In [1]:
import pandas as pd
import numpy as np
from functions import *

In [2]:
# set paths to data
path = '/Users/johan/Library/CloudStorage/GoogleDrive-johan.oelgaard@gmail.com/My Drive/04 Økonomi/10 Thesis/Data'

# read monthly market data from eikon
finacials = 'eikon_financials.xlsx'
eikon_dfs = pd.read_excel(path + '/' + finacials, sheet_name=None)
eikon_keys = eikon_dfs.keys()
# print(eikon_keys)

In [3]:
fin_df = eikon_dfs['financials'].iloc[:, 1:]

# set name for first two columns
fin_df.columns = ['timestamp', 'ticker'] + list(fin_df.columns[2:])

# set everything except the first two columns to numeric
for col in fin_df.columns[2:]:
    fin_df[col] = pd.to_numeric(fin_df[col], errors='coerce')

fin_df['fy_end'] = fin_df['timestamp'] + pd.DateOffset(years=1)

# set index to timestamp
fin_df.set_index('timestamp', inplace=True)


# calculate revenue
fin_df['revenue'] = (fin_df['Revenue']
                     .fillna(fin_df['Total Revenue'])
                     .fillna(fin_df['Bank Total Revenue']))
# set R&D to 0 if NaN
fin_df['rnd'] = fin_df['Research And Development'].fillna(0)

# calculate cash
fin_df['cash'] = (fin_df['Cash']
                  .fillna(fin_df['Cash and Equivalents'])
                  .fillna(fin_df['Cash and Short Term Investments'])
                  .fillna(fin_df['Cash & Due From Banks']))

fin_df['grossprofit'] = (fin_df['Gross Profit']
                         .fillna(fin_df['revenue']))


fin_df['opinc'] = (fin_df['Operating Income']
                   .fillna(fin_df['grossprofit']-fin_df['Total Operating Expense'])
                   .fillna(fin_df['Net Income Before Taxes']))

fin_df['cashflow'] = (fin_df['Cash Flow']
             .fillna(fin_df['Cash from Operating Activities, Cumulative']))


fin_df['debt'] = (fin_df['Total Debt']
                  .fillna(fin_df['Total Liabilities']))

fin_df['currentassets'] = fin_df['Total Current Assets'].fillna(0)

# drop columns
fin_df.drop(columns=['Revenue', 'Total Revenue', 'Bank Total Revenue',
                    'Research And Development', 
                    'Cash','Cash and Equivalents','Cash and Short Term Investments', 'Cash & Due From Banks',
                    'Gross Profit',
                    'Operating Income', 'Total Operating Expense',
                    'Cash Flow', 'Cash from Operating Activities, Cumulative',
                    'Total Debt', 'Total Liabilities',
                    'Total Current Assets',
                    ],inplace=True)

fin_df = fin_df.rename(columns={'Net Income Before Taxes': 'netinc','Total Assets, Reported': 'assets','Cash Flow':'cashflow'})

# display(fin_df.head(60))

# save to csv
fin_df.to_csv('data/financials.csv', index=False)